In [470]:
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim

import copy

torch.set_printoptions(threshold=float('inf'))

In [471]:
data = np.load('chr19.npz', allow_pickle=True)

In [472]:
#### Dataset Class
## Notice that the __init__ method contains an argument `apply_log10`, if you set it to True
## you will apply a log10 to the raw counts. We can experiment with this
class MethDataset(Dataset):
    def __init__(self, sequence, histone, methylation, coords, apply_log10=False):
        self.sequence = sequence
        self.histone = histone
        self.methylation = methylation
        self.transform = apply_log10
        self.coords = coords
        self.histone_names = ['H3K4me3', 'H3K36me2', 'H3K27me3', 'H3K9me3']

    def __len__(self):
        return self.methylation.shape[0]

    def __getitem__(self, idx):
        
        sequence = torch.from_numpy(self.sequence[idx])
        histone = self.histone.astype(np.float32)

        H3K4me3 = torch.from_numpy(histone[:, :, 0][idx].astype(np.float32)) if not self.transform else torch.from_numpy(np.log10(histone[:, :, 0]+1e-4)[idx])
        H3K36me2 = torch.from_numpy(histone[:, :, 1][idx].astype(np.float32)) if not self.transform else torch.from_numpy(np.log10(histone[:, :, 1]+1e-4)[idx])
        H3K27me3 = torch.from_numpy(histone[:, :, 2][idx].astype(np.float32)) if not self.transform else torch.from_numpy(np.log10(histone[:, :, 2]+1e-4)[idx])
        H3K9me3 = torch.from_numpy(histone[:, :, 3][idx].astype(np.float32)) if not self.transform else torch.from_numpy(np.log10(histone[:, :, 3]+1e-4)[idx])

        methylation = self.methylation[idx]
        coordinates = self.coords[idx]

        return sequence, H3K4me3, H3K36me2, H3K27me3, H3K9me3, methylation, coordinates

In [473]:
size = data['dna'].shape[0]
split_index = int(0.8 * size) ### 80% of the data will be for training

# I'm applying log10 in both cases
train_dataset = MethDataset(sequence = data['dna'][:split_index],
                           histone = data['histone'][:split_index], 
                           methylation = data['methyl'][:split_index],
                           coords = data['coords'][:split_index],
                           apply_log10=True)

test_dataset = MethDataset(sequence = data['dna'][split_index:],
                           histone = data['histone'][split_index:], 
                           methylation = data['methyl'][split_index:],
                           coords = data['coords'][split_index:],
                           apply_log10=True)

In [474]:
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [475]:
### Model class
## 1- My idea is to be able to control de architecture of the model, and training parameters since the model is created.
## This should make it easier to debug and to try different architectures, and the architecture of the model can be 
## specified through the arguments.
## 2- The `forward` method unsqueezes the input so the model understands the structure in batches.
## 3- There is a method called `training_loop`. Please, complete it, after you specify the architecture, add the loss function, and backward
## propagation step
## 4- I think we can add an `eval_loop` method, in which we iterate over the `test_dataloader` and evaluate the accuracy of the model (R^2)
## 5- Try some architectures, and some way to pass arguments to the model, such that we can try different numbers without having problems
## with tensor shapes and things like that. The idea is to be able to test certain combinations of numbers, so we can use Optuna to make
## a bayesian search for "optimal" parameters. Look at papers where people use CNNs for DNA and histone marks, try to have a similar architecture
## and let's start with that
class Model(nn.Module):
    def __init__(self, DNA_kernel_sizes, DNA_strides, DNA_conv_channels,
                    epochs=100, learning_rate=1e-3, optimizer=torch.optim.SGD):
        super().__init__()
        # Module parameters
        self.DNA_layer1_kernel_size, self.DNA_layer2_kernel_size, self.DNA_layer3_kernel_size, self.DNA_layer4_kernel_size = DNA_kernel_sizes
        self.DNA_conv_channels = DNA_conv_channels
        self.DNA_layer1_stride, self.DNA_layer2_stride, self.DNA_layer3_stride, self.DNA_layer4_stride = DNA_strides

        # Training parameters
        self.epochs = epochs
        self.learning_rate=learning_rate
        self.optimizer = optimizer
        
        # Modules and architecture
        self.dna_module = nn.Sequential(
            nn.Conv1d(in_channels=4, out_channels=DNA_conv_channels, kernel_size=(self.DNA_layer1_kernel_size), 
                        stride=self.DNA_layer1_stride, padding=0),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=(self.DNA_layer2_kernel_size), 
                        stride=self.DNA_layer2_stride, padding=0),
            nn.Conv1d(in_channels=DNA_conv_channels, out_channels=1, kernel_size=(self.DNA_layer3_kernel_size), 
                        stride=self.DNA_layer3_stride, padding=0),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=(self.DNA_layer4_kernel_size), 
                        stride=self.DNA_layer4_stride, padding=0)
        )

        self.histone_module = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=1, kernel_size=(self.DNA_layer1_kernel_size), 
                        stride=self.DNA_layer1_stride, padding=0),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=(self.DNA_layer2_kernel_size), 
                        stride=self.DNA_layer2_stride, padding=0),
            nn.Conv1d(in_channels=1, out_channels=1, kernel_size=(self.DNA_layer3_kernel_size), 
                        stride=self.DNA_layer3_stride, padding=0),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=(self.DNA_layer4_kernel_size), 
                        stride=self.DNA_layer4_stride, padding=0)
        )

        self.H3K4me3_module = self.histone_module
        self.H3K36me2_module = self.histone_module
        self.H3K27me3_module = self.histone_module
        self.H3K9me3_module = self.histone_module


        self.attn = nn.MultiheadAttention(embed_dim=5, num_heads=5, batch_first=True)

        self.fc = nn.Sequential(
            nn.Linear(5, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.Linear(10, 1),
            nn.Softplus()
        )

    def forward(self, sequence, H3K4me3, H3K36me2, H3K27me3, H3K9me3, methylation):
        sequence = sequence.to(torch.float32).permute(0, 2, 1) ### Changed to (B,C=4,L=500) to use Conv1D
        dna_module_output = self.dna_module(sequence).reshape(-1, 1)

        H3K4me3_module_output = self.H3K4me3_module(H3K4me3.unsqueeze(1)).reshape(-1, 1)
        H3K36me2_module_output = self.H3K36me2_module(H3K36me2.unsqueeze(1)).reshape(-1, 1)
        H3K27me3_module_output = self.H3K27me3_module(H3K27me3.unsqueeze(1)).reshape(-1, 1)
        H3K9me3_module_output = self.H3K9me3_module(H3K9me3.unsqueeze(1)).reshape(-1, 1)
        
        # stack = torch.stack([dna_module_output, H3K4me3_module_output, H3K36me2_module_output, H3K27me3_module_output, H3K9me3_module_output]).permute(1,0,2) # Not sure if this is ok
        stack = torch.stack([dna_module_output, H3K4me3_module_output, H3K36me2_module_output, H3K27me3_module_output, H3K9me3_module_output]).permute(1,0,2) # Not sure if this is ok
        prediction = self.fc(stack.squeeze(2))
        
        return prediction


    def training_loop(self, loss_fn, train_dataset, batch_size=10):
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        optimizer = self.optimizer(self.parameters(), lr=self.learning_rate)
        loss_fn = loss_fn()

        self.train()
        for e in range(self.epochs):
            for i, (sequence, H3K4me3, H3K36me2, H3K27me3, H3K9me3, methylation, coordinates) in enumerate(train_dataloader):
                
                prediction = self.forward(sequence, H3K4me3, H3K36me2, H3K27me3, H3K9me3, methylation)

                loss = loss_fn(prediction, methylation.unsqueeze(-1).to(torch.float32))
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()


            if e % 50 == 0:
                print(f"Iter: {e+1}, Loss: {loss.item()}")
    
    def eval_loop(args, kwargs):
        pass


In [477]:
model = Model(DNA_kernel_sizes=(25,5,5,3), DNA_strides=(5,5,5,1), DNA_conv_channels = 2,
                    epochs=150, learning_rate=1e-3, optimizer=torch.optim.Adam)

In [ ]:
model.training_loop(loss_fn=nn.MSELoss, train_dataset=train_dataset, batch_size=10)

In [179]:
sequence, H3K4me3, H3K36me2, H3K27me3, H3K9me3, methylation, coordinates = next(iter(train_dataloader))

In [190]:
print(H3K4me3.shape)
print(H3K4me3.unsqueeze(1).shape)

torch.Size([8, 500])
torch.Size([8, 1, 500])


In [104]:
seq[0].shape    # [8, 500, 4] I need to reshape this to (B,C=4,L=500)
seq[0].permute(0,2, 1)    # [8, 500, 4]

tensor([[[1, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 1, 1, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0]],

        [[0, 0, 1,  ..., 1, 0, 1],
         [0, 1, 0,  ..., 0, 1, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 1],
         [0, 1, 1,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 1, 0, 0]],

        [[1, 0, 1,  ..., 1, 0, 1],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 1,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 1],
         [1, 0, 0,  ..., 1, 1, 0]]])

In [98]:
seq[0][0].transpose(-2,1)

tensor([[1, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 1, 1, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0]])

In [86]:
seq[0].permute(0, 2, 1)[0]#.shape

tensor([[1, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 1, 1, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0]])

In [53]:
[
    [[1,
      0,
      0,
      1]
     ],
     [[0,0,1,0],
      [0,0,0,0],
      [0,0,0,0],
      [0,0,1,0]],
     [],
     [],CHANNEL4], BATCH1
    [],
    [],
    [],
    [],
 ]
 BATCH=5, CHANNEL=4, WIDTH=, HEIGHT=

NameError: name 'self' is not defined

In [152]:
def get_out_Conv1D(length, kernel_size,
                  padding, stride, dilation=1):
    return np.floor((length+2*padding-dilation*(kernel_size-1)-1)/stride+1).astype(int)

def get_out_MaxPool1D(length, kernel_size,
                  padding, stride, dilation=1):
    return np.floor((length+2*padding-dilation*(kernel_size-1)-1)/stride+1).astype(int)


100

In [163]:
k1=25
s1=5

k2=5
s2=5

k3=5
s3=5

k4=3
s4=1

size1 = get_out_Conv1D(length=500, kernel_size=k1, padding=0, stride=s1)
size2 = get_out_MaxPool1D(length=size1, kernel_size=k2 ,padding=0, stride=s2, dilation=1)
size3 = get_out_Conv1D(length=size2, kernel_size=k3, padding=0, stride=s3)
size4 = get_out_MaxPool1D(length=size3, kernel_size=k4, padding=0, stride=s4, dilation=1)


print(size4)

1


In [47]:
w2, h2

(0, 18)

In [159]:
lr = 1e-3
optim = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
## GPyOpt check this library (== Optuna)

### Model from 

# def d_cnn_model(input_length):
#     model = Sequential()

#     model.add(Dropout(0.2, input_shape=(input_length,1)))
#     model.add(Conv1D(32, 3, activation='relu'))
#     # model.add(Conv1D(32, 3, activation='relu'))
#     # model.add(Dropout(0.5))
#     model.add(MaxPooling1D(2))

#     model.add(Conv1D(64, 3, activation='relu'))
#     # # model.add(Dropout(0.5))
#     model.add(MaxPooling1D(2))

#     model.add(Conv1D(128, 3, activation='relu'))
#     # # model.add(Dropout(0.5))
#     model.add(MaxPooling1D(2))